In [1]:
import requests
import lxml.html

In [2]:
url="https://declarator.org/office/449"
root="https://declarator.org"

In [3]:
# This requires the installation of Splash
r=requests.get("http://localhost:8050/render.html",params={"url":url})

In [4]:
html=lxml.html.fromstring(r.content)

In [5]:
first_scrape=[]
for element in html.xpath('//tr'):
    year=int(element.xpath('./td[position()=1]/text()')[0])
    for element_2 in element.xpath('./td[position()=2]//ul[@class="list-inline activetext"]/li/a'):
        if element_2.xpath('./text()')[0].strip()=="Anti-corruption declaration":
            anti_corrupt_url=root+element_2.xpath("./@href")[0]
            # Get details
            r2=requests.get("http://localhost:8050/render.html",params={'url':anti_corrupt_url})
            html2=lxml.html.fromstring(r2.content)
            element_final=html2.xpath('//td[@colspan="3" or @colspan="4"]')[-1]
            text=element_final.xpath(".//p//text()")
            # Get official name
            element_person=html2.xpath('//td[@class="person"]')[-1]
            name=element_person.xpath('./div[1]//text()')
            title=element_person.xpath('.//div[2]//text()')
            new_url=element_person.xpath('./div[@class="person_name activetext"]/a/@href')
            new_url=root+new_url[0]
            first_scrape.append((year,text,name,title,new_url))

In [6]:
import re

def clean_text(list_of_words):
    clean_list=[re.sub(r"\n","",word).strip() for word in list_of_words]
    clean_list=[re.sub(r" m"," sqm",word).strip() for word in clean_list]
    clean_list=[item for item in clean_list if len(item)>0 and item!="2"]
    return clean_list

def clean_text_details(list_of_words):
    clean_list=clean_text(list_of_words)
    new_list=[]
    i=0
    while i <=len(clean_list)-1:
        if i==len(clean_list)-1:
            new_list.append(clean_list[i])
            i+=1
        elif clean_list[i+1][0]=="(" or clean_list[i+1][0]==",":
            new_list.append(clean_list[i]+" "+clean_list[i+1])
            i+=2
        else:
            new_list.append(clean_list[i])
            i+=1
    return new_list

def clean_text_name(list_of_words):
    clean_list=clean_text(list_of_words)
    #full_name=", ".join(clean_list)
    return clean_list

def clean_text_title(list_of_words):
    clean_list=clean_text(list_of_words)
    full_title=" ".join(clean_list)
    return full_title

In [7]:
final=[(item_1,clean_text_details(item_2),clean_text_name(item_3),clean_text_title(item_4),item_5) for item_1,item_2,item_3,item_4,item_5 in first_scrape]

In [8]:
def create_dicts(year,cleaned_list,name,title,url):
    index_list=[]
    for item in ["Income","Real estate","Transport","Accounts"]:
        try:
            index=cleaned_list.index(item)
            index_list.append(index)
        except:
            continue
    final_output=[]
    index_list.append(len(cleaned_list))
    for i in range(len(index_list)-1):
        begin=index_list[i]+1
        end=index_list[i+1]
        span=cleaned_list[begin:end]
        for item in span:
            return_list={}
            return_list['last_name']=name[0]
            return_list['first_name']=name[1]
            return_list["year"]=year
            return_list["link"]=url
            return_list["position"]=title
            return_list["asset_type"]=cleaned_list[index_list[i]]
            return_list["asset_value"]=item
            final_output.append(return_list)  
    return final_output

In [9]:
final_2=[create_dicts(item_1,item_2,item_3,item_4,item_5) for item_1,item_2,item_3,item_4,item_5 in final]

In [10]:
final_3=[]
for final_element in final_2:
    final_3.extend(final_element)

In [11]:
import pandas as pd
output_df=pd.DataFrame(final_3)

In [12]:
output_df

,last_name,first_name,year,link,position,asset_type,asset_value
0,Putin,Vladimir Vladimirovich,2020,https://declarator.org/person/582/,The President Of The Russian Federation,Income,"9 994 692,00 руб."
1,Putin,Vladimir Vladimirovich,2020,https://declarator.org/person/582/,The President Of The Russian Federation,Real estate,"Apartment, 77 sqm"
2,Putin,Vladimir Vladimirovich,2020,https://declarator.org/person/582/,The President Of The Russian Federation,Real estate,"Apartment, 153.7 sqm (in use )"
3,Putin,Vladimir Vladimirovich,2020,https://declarator.org/person/582/,The President Of The Russian Federation,Real estate,"Garage, 18 sqm"
4,Putin,Vladimir Vladimirovich,2020,https://declarator.org/person/582/,The President Of The Russian Federation,Real estate,"Гаражное место, 18 sqm (in use )"
...,...,...,...,...,...,...,...
117,Medvedev,Dmitrii Anatolevich,2008,https://declarator.org/person/542/,The President of the Russian Federation,Real estate,"Супруг(а): Garage, Moscow, 16.2 sqm , 1 доли"
118,Medvedev,Dmitrii Anatolevich,2008,https://declarator.org/person/542/,The President of the Russian Federation,Real estate,"Супруг(а): Garage, Moscow, 16.3 sqm , 1 доли"
119,Medvedev,Dmitrii Anatolevich,2008,https://declarator.org/person/542/,The President of the Russian Federation,Transport,"Супруг(а): Passenger car, Volkswagen Golf 199..."
120,Medvedev,Dmitrii Anatolevich,2008,https://declarator.org/person/542/,The President of the Russian Federation,Accounts,"135 144,05 руб. (1 счет)"


The code below is to extract foreign name

In [13]:
def find_russian_name(x):
    root_person="http://declarator.org/person_report/"
    id_person=re.findall(r"[0-9]+",x)[0]
    full_id_url=root_person+id_person+"/"
    df_name=pd.read_excel(full_id_url)
    return df_name['ФИО'].unique()[0]

In [14]:
list_of_russian_names=[find_russian_name(link) for link in output_df.link.unique()]

In [15]:
df_russian_name=pd.DataFrame({"id":output_df.link.unique(),"russian_name":list_of_russian_names})

In [16]:
output_df_2=output_df.merge(df_russian_name,how="left",right_on="id",left_on="link").drop(['id'],axis=1)

In [32]:
output_df_2.to_excel("Russia_president_anti_corruption.xlsx",index=False)

In [17]:
output_df_2.to_csv("Russia_president_anti_corruption.csv",index=False)